In [1]:
import pandas as pd
import cv2
import os
import numpy as np
import time
import matplotlib.pyplot as plt
%matplotlib inline
import json
import pickle
# import tensorflow as tf
import keras
import keras.backend as K
# from keras_retinanet.models.resnet import custom_objects
# from keras_retinanet.models.resnet import resnet_retinanet as retinanet, custom_objects, download_imagenet
# import keras_retinanet
# import keras_retinanet.bin.train
from sklearn.utils import shuffle 
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [2]:
json_file = open('/home/jovyan/hico_model.json', 'r')
model_json = json_file.read()
json_file.close()
model = keras.models.model_from_json(model_json)
model.load_weights('/home/jovyan/hico_second_try.h5')

In [3]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None, 4)      0                                            
__________________________________________________________________________________________________
lambda_5 (Lambda)               (None, None, 2, 4)   0           input_8[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 800, 1200, 3) 0                                            
__________

In [3]:
train_data = pd.read_json('../train_data_simple.json')
train_data = train_data.sort_index()
retinanet_train_data = pd.read_json('retinanet_train_data.json')
retinanet_train_data = retinanet_train_data.sort_index()
with open('../hoi_dict.json', 'r') as f:
    hoi_dict = json.load(f)
with open('../obj_dict.json', 'r') as f:
    obj_dict = json.load(f)
obj_word_to_id = {v:int(k) for k,v in obj_dict.items()}

In [9]:
def train_data_gen(retinanet_train_data,train_data,id_):
    img_stack = np.array([]).reshape(0,800,1200,3)
    human_stack = np.array([]).reshape(0,4)
    object_stack = np.array([]).reshape(0,4)
    object_class_stack = np.array([]).reshape(1,0)
            #image
    img = cv2.imread(os.path.join('/home/jovyan/projectdata/cht01/hico_20160224_det/images/train2015/',train_data['name'][id_]))
    img = cv2.resize(img, (1200,800))
    img = img/255
    img = np.expand_dims(img,axis=0)
    img_stack = np.row_stack([img_stack,img])

            
            #human object bbox pair & object class
    for i in range(len(retinanet_train_data.human_boxes[id_])):
        if (np.max(retinanet_train_data.human_scores[id_]) >0.5) & (retinanet_train_data.human_scores[id_][i]<0.5):
            continue
        ho_pair_h = retinanet_train_data.human_boxes[id_][i]
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
#         ho_pair_h = np.expand_dims(ho_pair_h,axis=0)
        human_stack = np.row_stack([human_stack,ho_pair_h])

    for j in range(len(retinanet_train_data.obj_boxes[id_])):
        if (np.max(retinanet_train_data.obj_scores[id_]) >0.5) & (retinanet_train_data.obj_scores[id_][j]<0.5):
            continue
#         print(j)
        ho_pair_o = retinanet_train_data.obj_boxes[id_][j]
        object_class = retinanet_train_data.obj_classes[id_][j]
        ho_pair_o = np.expand_dims(ho_pair_o,axis=0)
#       ho_pair_o = np.expand_dims(ho_pair_o,axis=0)

#       object_class = np.expand_dims(object_class,axis=0)
            
#       img_stack = np.row_stack([img_stack,img])
        object_stack = np.row_stack([object_stack,ho_pair_o])
        object_class_stack = np.column_stack([object_class_stack,object_class])
#     img_stack = np.expand_dims(img_stack,axis=0)
    human_stack = np.expand_dims(human_stack,axis=0)
    object_stack = np.expand_dims(object_stack,axis=0)
#     object_class_stack = np.expand_dims(object_class_stack,axis=0)
    train_batch = [img_stack,human_stack,object_stack,object_class_stack]
    return train_batch

In [10]:
test_one_img = train_data_gen(retinanet_train_data,train_data,1)
test_one_img[0].shape,test_one_img[1].shape,test_one_img[2].shape,test_one_img[3].shape

((1, 800, 1200, 3), (1, 1, 4), (1, 8, 4), (1, 8))

In [12]:
pred_all = []
for i in tqdm(range(train_data.shape[0])):
    train_one_img = train_data_gen(retinanet_train_data,train_data,i)
    if train_one_img[1].any()==True:
        pred = model.predict(train_one_img)
    else:
        pred = [np.array([]).reshape((1,0,600)),np.array([]).reshape((1,0))]
    pred_all.append(pred)

HBox(children=(IntProgress(value=0, max=26561), HTML(value='')))

In [20]:
with open('training_predict_result','wb') as file:
    pickle.dump(pred_all,file)

# mAP calculation

In [4]:
with open('training_predict_result','rb') as file:
    pred_all = pickle.load(file)

In [5]:
retinanet_train_data.head()

,human_boxes,human_boxes_scale,human_classes,human_scores,obj_boxes,obj_boxes_scale,obj_classes,obj_scores
0,"[[408.5285644531, 137.1702575684, 590.03033447...","[[217, 82, 314, 289], [189, 108, 388, 279], [2...","[0, 0, 0, 0]","[0.24128636720000002, 0.2087177634, 0.35492742...","[[384.8578491211, 70.9428253174, 883.437683105...","[[205, 42, 471, 217], [217, 82, 314, 289], [16...","[3, 0, 3, 3, 3, 0, 0, 0, 3]","[0.2452710122, 0.24128636720000002, 0.49595344..."
1,"[[515.2573242188, 2.3874969482, 1107.571899414...","[[274, 1, 590, 126], [401, 0, 636, 140]]","[0, 0]","[0.34479168060000004, 0.6770228148]","[[756.3471679688, 567.7899169922, 817.62744140...","[[403, 340, 436, 363], [405, 341, 436, 363], [...","[49, 51, 51, 39, 41, 39, 55, 53, 45, 41, 45, 7...","[0.2125296295, 0.235072419, 0.2925423980000000..."
2,[],[],[],[],"[[295.9039916992, 352.1661376953, 341.69757080...","[[147, 281, 170, 341], [0, 361, 15, 388], [0, ...","[9, 2, 2, 2, 2, 2, 2, 2, 2, 7, 5, 5, 5]","[0.45125731830000004, 0.2788699865, 0.64107280..."
3,"[[939.8218383789, 401.7608642578, 984.80657958...","[[501, 214, 525, 277], [130, 203, 165, 311], [...","[0, 0, 0, 0, 0, 0]","[0.6018131971, 0.2301999032, 0.3746683598, 0.9...","[[939.8218383789, 401.7608642578, 984.80657958...","[[501, 214, 525, 277], [130, 203, 165, 311], [...","[0, 0, 27, 0, 0, 0, 0]","[0.6018131971, 0.2301999032, 0.2251143306, 0.3..."
4,"[[192.0525970459, 291.7764892578, 370.51116943...","[[102, 155, 197, 265]]",[0],[0.9221789241],"[[321.814666748, 445.8058776855, 350.372467041...","[[171, 236, 186, 261], [532, 275, 554, 292], [...","[41, 67, 39, 0, 62, 59, 60, 60, 13, 60, 59]","[0.21376061440000002, 0.2884043753, 0.73071849..."


In [6]:
#先跑object loop再跑human loop
#bbox = (x1,y1,x2,y2)
def bbox_gen(retinanet_train_data,id_):
    human_stack = np.array([]).reshape(0,4)
    object_stack = np.array([]).reshape(0,4)
    object_class_stack = np.array([]).reshape(0,1)

    for i in range(len(retinanet_train_data.human_boxes_scale[id_])):
        if (np.max(retinanet_train_data.human_scores[id_]) >0.5) & (retinanet_train_data.human_scores[id_][i]<0.5):
            continue
        ho_pair_h = retinanet_train_data.human_boxes_scale[id_][i]
        ho_pair_h = np.expand_dims(ho_pair_h,axis=0)

        for j in range(len(retinanet_train_data.obj_boxes_scale[id_])):
            if (np.max(retinanet_train_data.obj_scores[id_]) >0.5) & (retinanet_train_data.obj_scores[id_][j]<0.5):
                continue
            ho_pair_o = retinanet_train_data.obj_boxes_scale[id_][j]
            ho_pair_o = np.expand_dims(ho_pair_o,axis=0)
            
            object_class = retinanet_train_data.obj_classes[id_][j]
            object_class = np.expand_dims(object_class,axis=0)
            
            human_stack = np.row_stack([human_stack,ho_pair_h])
            object_stack = np.row_stack([object_stack,ho_pair_o])
            object_class_stack = np.row_stack([object_class_stack,object_class])
    object_class_stack = np.squeeze(object_class_stack)
    
    ho_bbox = [human_stack,object_stack,object_class_stack]
    return ho_bbox

In [8]:
result_action = []
result_h_bbox =[]
result_o_bbox = []
result_o_class = []
for i in tqdm(range(len(retinanet_train_data))):
    temp = bbox_gen(retinanet_train_data,i)
    result_action.append(pred_all[i][0][0])
    result_h_bbox.append(temp[0])
    result_o_bbox.append(temp[1])
    result_o_class.append(temp[2])

HBox(children=(IntProgress(value=0, max=26561), HTML(value='')))

In [9]:
id_=0
# id_+=1
print(id_)
result_action[id_].shape,result_h_bbox[id_].shape,result_o_bbox[id_].shape,result_o_class[id_].shape

0


((2, 600), (2, 4), (2, 4), (2,))

In [11]:
train_data.head()

,action_no,human_bbox,name,obj_id,obj_list,object_bbox,size
0,"[153, 154, 155, 156]","[209, 434, 28, 320]",HICO_train2015_00000001.jpg,[3],[motorcycle],"[64, 576, 103, 399]","[640, 480, 3]"
1,[224],"[262, 638, 1, 138]",HICO_train2015_00000002.jpg,[46],[banana],"[162, 258, 131, 165]","[640, 480, 3]"
2,[65],"[567, 580, 361, 390]",HICO_train2015_00000003.jpg,[5],[bus],"[74, 510, 52, 484]","[599, 640, 3]"
3,"[48, 49]","[222, 484, 29, 370]",HICO_train2015_00000004.jpg,[39],[bottle],"[265, 365, 87, 122]","[640, 427, 3]"
4,[242],"[82, 203, 154, 272]",HICO_train2015_00000005.jpg,[59],[bed],"[1, 313, 176, 422]","[640, 425, 3]"


In [26]:
np.array(train_data.human_bbox[id_])[[0,2,1,3]]

array([209,  28, 434, 320])

In [15]:
np.array([[item[0],item[2],item[1],item[3]] for item in np.array(train_data.human_bbox[id_])])

IndexError: invalid index to scalar variable.

In [28]:
#bbox = (x1,x2,y1,y2) >>> (x1,y1,x2,y2)
gt_action = []
gt_h_bbox = []
gt_o_bbox = []
gt_o_class = []
for id_ in tqdm(range(len(train_data))):
    gt_action_sublist = []
    gt_h_bbox_sublist = []
    gt_o_bbox_sublist = []
    h_box = np.array(train_data.human_bbox[id_])[[0,2,1,3]]
    o_box = np.array(train_data.object_bbox[id_])[[0,2,1,3]]
    action_no = np.array(train_data.action_no[id_])
    for i in action_no:
        gt_action_sublist.append(i)
        gt_h_bbox_sublist.append(h_box)
        gt_o_bbox_sublist.append(o_box)
    
    gt_action.append(np.array(gt_action_sublist))
    gt_h_bbox.append(np.array(gt_h_bbox_sublist))
    gt_o_bbox.append(np.array(gt_o_bbox_sublist))
    gt_o_class.append(np.array(gt_o_class_sublist))


HBox(children=(IntProgress(value=0, max=26561), HTML(value='')))

In [30]:
id_=0
# id_+=1
print(id_)
gt_action[id_].shape,gt_h_bbox[id_].shape,gt_o_bbox[id_].shape

0


((4,), (4, 4), (4, 4))

In [31]:
# (x1, y1, x2, y2)
def bb_iou(boxA,boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
    if boxA[0]>boxB[2] or boxA[2]<boxB[0] or boxA[1]>boxB[3] or boxA[3]<boxB[1]:
        iou = 0
    else:
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])
        # compute the area of intersection rectangle
        interArea = (xB - xA + 1) * (yB - yA + 1)
 
        # compute the area of both the prediction and ground-truth
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
 
        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)
 
    # return the intersection over union value
    return iou

In [33]:
#first_try
AP_list = []
for id_ in tqdm(range(len(result_h_bbox))):
# for id_ in tqdm(range(1,2)):   
    AP_sublist = []
    for action_id in gt_action[id_]:
        gt_1action_h_bbox = gt_h_bbox[id_][gt_action[id_]==action_id]
        gt_1action_o_bbox = gt_o_bbox[id_][gt_action[id_]==action_id]
        gt_1action_action = gt_action[id_][gt_action[id_]==action_id]
        gt_match = np.zeros([gt_1action_h_bbox.shape[0]])
        pred_match = np.zeros([result_h_bbox[id_].shape[0]])
        indices = np.argsort(result_action[id_][:,action_id-1])[::-1]
        score = result_action[id_][:,action_id-1][indices]
        h_bbox = result_h_bbox[id_][indices]
        o_bbox = result_o_bbox[id_][indices]
        for i in range(len(h_bbox)):
            for j in range(len(gt_1action_h_bbox)):
                iou_h = bb_iou(h_bbox[i],gt_1action_h_bbox[j])
                iou_o = bb_iou(o_bbox[i],gt_1action_o_bbox[j])
                if np.min([iou_h,iou_o])>0.5:
                    if gt_match[j]==1:
                        continue
                    pred_match[i]=1
                    gt_match[j]=1
        precisions = np.cumsum(pred_match) / (np.arange(len(pred_match)) + 1)
        recalls = np.cumsum(pred_match).astype(np.float32) / len(gt_match)
        precisions = np.concatenate([[0], precisions, [0]])
        recalls = np.concatenate([[0], recalls, [1]])

        for i in range(len(precisions) - 2, -1, -1):
            precisions[i] = np.maximum(precisions[i], precisions[i + 1])
        indice = np.where(recalls[:-1] != recalls[1:])[0] + 1
        AP = ((recalls[indice] - recalls[indice - 1]) *precisions[indice])[0]
        AP_sublist.append(AP)
    AP_list.append(AP_sublist)

HBox(children=(IntProgress(value=0, max=26561), HTML(value='')))

In [34]:
AP_list = np.array(AP_list)
temp = [item for sublist in AP_list for item in sublist]
temp = np.array(temp)
temp[np.isnan(temp)]=0
np.mean(temp)

0.32799301349994692

In [36]:
#copycat
AP_list = []
for id_ in tqdm(range(len(result_h_bbox))):   
# for id_ in tqdm(range(1,2)):   
    AP_sublist = []
    for action_id in gt_action[id_]:
        gt_1action_h_bbox = gt_h_bbox[id_][gt_action[id_]==action_id]
        gt_1action_o_bbox = gt_o_bbox[id_][gt_action[id_]==action_id]
        gt_1action_action = gt_action[id_][gt_action[id_]==action_id]
        tp = np.zeros([result_h_bbox[id_].shape[0]])
        fp = np.zeros([result_h_bbox[id_].shape[0]])
        npos = gt_1action_action.shape[0]
        indices = np.argsort(result_action[id_][:,action_id-1])[::-1]
        score = result_action[id_][:,action_id-1][indices]
        h_bbox = result_h_bbox[id_][indices]
        o_bbox = result_o_bbox[id_][indices]
        for i in range(len(h_bbox)):
            for j in range(len(gt_1action_h_bbox)):
                iou_h = bb_iou(h_bbox[i],gt_1action_h_bbox[j])
                iou_o = bb_iou(o_bbox[i],gt_1action_o_bbox[j])
                if np.min([iou_h,iou_o])>0.5:
                    tp[i]=1
                else:
                    fp[i]=1
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp/npos
        prec = tp/(fp+tp)
        AP=0
        for t in np.arange(0.0, 1.0, 0.1):
            if prec[rec>=t].any():
                p = np.max(prec[rec>=t])
                AP = AP +p/11
            else:
                p=0
                AP = AP +p/11
        AP_sublist.append(AP)
    AP_list.append(AP_sublist)

HBox(children=(IntProgress(value=0, max=26561), HTML(value='')))

In [37]:
AP_list = np.array(AP_list)
temp = [item for sublist in AP_list for item in sublist]
temp = np.array(temp)
temp[np.isnan(temp)]=0
np.mean(temp)

0.29930175855468344

In [38]:
#600 cat
AP_list = []
for id_ in tqdm(range(len(result_h_bbox))):   
# for id_ in tqdm(range(1,2)):   
    AP_sublist = []
    for action_id in range(1,601):
        gt_1action_h_bbox = gt_h_bbox[id_][gt_action[id_]==action_id]
        gt_1action_o_bbox = gt_o_bbox[id_][gt_action[id_]==action_id]
        gt_1action_action = gt_action[id_][gt_action[id_]==action_id]
        tp = np.zeros([result_h_bbox[id_].shape[0]])
        fp = np.zeros([result_h_bbox[id_].shape[0]])
        npos = gt_1action_action.shape[0]
        indices = np.argsort(result_action[id_][:,action_id-1])[::-1]
        score = result_action[id_][:,action_id-1][indices]
        h_bbox = result_h_bbox[id_][indices]
        o_bbox = result_o_bbox[id_][indices]
        for i in range(len(h_bbox)):
            for j in range(len(gt_1action_h_bbox)):
                iou_h = bb_iou(h_bbox[i],gt_1action_h_bbox[j])
                iou_o = bb_iou(o_bbox[i],gt_1action_o_bbox[j])
                if np.min([iou_h,iou_o])>0.5:
                    tp[i]=1
                else:
                    fp[i]=1
        fp = np.cumsum(fp)
        tp = np.cumsum(tp)
        rec = tp/npos
        prec = tp/(fp+tp)
        AP=0
        for t in np.arange(0.0, 1.0, 0.1):
            if prec[rec>=t].any():
                p = np.max(prec[rec>=t])
                AP = AP +p/11
            else:
                p=0
                AP = AP +p/11
        AP_sublist.append(AP)
    AP_list.append(AP_sublist)

HBox(children=(IntProgress(value=0, max=26561), HTML(value='')))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in greater_equal


In [39]:
AP_list = np.array(AP_list)
temp = [item for sublist in AP_list for item in sublist]
temp = np.array(temp)
temp[np.isnan(temp)]=0
np.mean(temp)

0.00089839733206717855

In [40]:
temp2 = [np.mean(item) for item in AP_list]
temp2 = np.array(temp2)
temp2[np.isnan(temp2)]=0
np.mean(temp2)

0.00089839733206718072